In [ ]:
import pandas as pd
from pycaret.classification import *
import seaborn as sns
import matplotlib.pyplot as plt

# 1. Cargar y explorar la data
df = pd.read_csv('caso2_microcredito_default.csv')

# 2. Análisis exploratorio NO trivial: identificar patrones por zona
sns.barplot(x='zona', y='default', data=df)
plt.title("Tasa de default por zona")
plt.show()

# 3. Segmentar por negocios donde el default es mayor
df_seg = df[df['tipo_negocio'].isin(['Agrícola', 'Comercio'])]

# 4. Configurar PyCaret con enfoque segmentado
exp = setup(data=df_seg,
            target='default',
            session_id=999,
            categorical_features=['zona', 'tipo_negocio'],
            ignore_features=['cuotas_pagadas'],
            remove_multicollinearity=True,
            multicollinearity_threshold=0.8,
            normalize=True)

# 5. Buscar el mejor modelo considerando AUC y Recall
best = compare_models(sort='Recall')

# 6. Evaluación de sensibilidad para reducir falsos negativos
final_model = tune_model(best, optimize='Recall')
plot_model(final_model, plot='confusion_matrix')
plot_model(final_model, plot='feature')

# 7. Interpretación por zona
interpret_model(final_model, plot='summary')

# 8. Simular nuevo crédito (no usar todo el df)
nuevo_cliente = pd.DataFrame({
    'zona': ['Selva'],
    'tipo_negocio': ['Agrícola'],
    'monto_credito': [4500],
    'mujer_emprendedora': [1],
    'hijos': [3]
})
pred = predict_model(final_model, data=nuevo_cliente)
print(pred)

# 9. Guardar para pipeline rural
save_model(final_model, 'modelo_default_rural_selva')